## Homework

The goal of this homework is to train a simple model for predicting the duration of a ride - similar to what we did in this module.

## Q1. Downloading the data

In [2]:
import pandas as pd

In [2]:
#pip install pyarrow

In [3]:
jan_url = 'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2022-01.parquet'
feb_url = 'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2022-02.parquet'
df_jan = pd.read_parquet(jan_url)
df_feb = pd.read_parquet(feb_url)
#print(df.head())


In [5]:
# Read the data for January. How many columns are there?
df_jan.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2463931 entries, 0 to 2463930
Data columns (total 19 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   VendorID               int64         
 1   tpep_pickup_datetime   datetime64[ns]
 2   tpep_dropoff_datetime  datetime64[ns]
 3   passenger_count        float64       
 4   trip_distance          float64       
 5   RatecodeID             float64       
 6   store_and_fwd_flag     object        
 7   PULocationID           int64         
 8   DOLocationID           int64         
 9   payment_type           int64         
 10  fare_amount            float64       
 11  extra                  float64       
 12  mta_tax                float64       
 13  tip_amount             float64       
 14  tolls_amount           float64       
 15  improvement_surcharge  float64       
 16  total_amount           float64       
 17  congestion_surcharge   float64       
 18  airport_fee           

In [6]:
jan_rows = df_jan.shape[0]
jan_rows

2463931

In [7]:
df_jan.shape

(2463931, 19)

In [8]:
df_jan.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,1,2022-01-01 00:35:40,2022-01-01 00:53:29,2.0,3.80,1.0,N,142,236,1,14.5,3.0,0.5,3.65,0.0,0.3,21.95,2.5,0.0
1,1,2022-01-01 00:33:43,2022-01-01 00:42:07,1.0,2.10,1.0,N,236,42,1,8.0,0.5,0.5,4.00,0.0,0.3,13.30,0.0,0.0
2,2,2022-01-01 00:53:21,2022-01-01 01:02:19,1.0,0.97,1.0,N,166,166,1,7.5,0.5,0.5,1.76,0.0,0.3,10.56,0.0,0.0
3,2,2022-01-01 00:25:21,2022-01-01 00:35:23,1.0,1.09,1.0,N,114,68,2,8.0,0.5,0.5,0.00,0.0,0.3,11.80,2.5,0.0
4,2,2022-01-01 00:36:48,2022-01-01 01:14:20,1.0,4.30,1.0,N,68,163,1,23.5,0.5,0.5,3.00,0.0,0.3,30.30,2.5,0.0


## Q2. Computing duration

In [12]:
# What's the standard deviation of the trips duration in January?
df_jan['duration'] = df_jan.tpep_dropoff_datetime - df_jan.tpep_pickup_datetime
df_jan.duration = df_jan.duration.apply(lambda td: td.total_seconds() / 60)
df_jan['duration'].std()

46.44530513776802

## Q3. Dropping outliers

In [14]:
# What fraction of the records left after you dropped the outliers?
df_jan = df_jan[(df_jan.duration >= 1) & (df_jan.duration <= 60)]
(df_jan.shape[0]/jan_rows)* 100

98.27547930522405

## Q4. One-hot encoding

In [15]:
# What's the dimensionality of this matrix (number of columns)?
from sklearn.feature_extraction import DictVectorizer

In [16]:
categorical = ['PULocationID', 'DOLocationID']
dict_features = df_jan[categorical].astype(str).to_dict(orient="records")

dv = DictVectorizer()
X_train = dv.fit_transform(dict_features)

In [17]:
X_train.shape

(2421440, 515)

In [18]:
print(f"Number of explicitly-stored values={X_train.getnnz()}")
print(f"Number of rows={X_train.shape[0]}")
print(f"Number of columns={X_train.shape[1]}")

Number of explicitly-stored values=4842880
Number of rows=2421440
Number of columns=515


## Q5. Training a model

In [19]:
# What's the RMSE on train?
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [21]:
target = 'duration'
y_train = df_jan[target].values

linear_reg = LinearRegression()
linear_reg.fit(X_train, y_train)

LinearRegression()

In [22]:
y_pred = linear_reg.predict(X_train)
mean_squared_error(y_train, y_pred, squared=False)

6.986190135982442

## Q6. Evaluating the model

In [23]:
# What's the RMSE on validation?
df_feb['duration'] = (df_feb.tpep_dropoff_datetime - df_feb.tpep_pickup_datetime).dt.total_seconds() / 60

In [24]:
df_feb = df_feb[(df_feb.duration >= 1) & (df_feb.duration <= 60)]

In [25]:
valid_dicts = df_feb[categorical].astype("str").to_dict(orient="records")
X_valid = dv.transform(valid_dicts)

In [26]:
X_valid

<2918187x515 sparse matrix of type '<class 'numpy.float64'>'
	with 5836368 stored elements in Compressed Sparse Row format>

In [27]:
y_valid = df_feb["duration"]

In [28]:
y_pred = lr.predict(X_valid)
mean_squared_error(y_valid, y_pred, squared=False)

7.786389489670289